[Подсчет строк](#Подсчет-строк)  
[searchRus](#searchRus)  
[](#)  
[](#)  
[](#)  


#### Подсчет строк 
всего в файлах cpp, h, ui  
в директории .\\a01diagcpp\\

In [26]:
import codecs
import os
import re

CODEC_FROM = "cp1251"
CODEC_TO = "utf-8"


class ClFind:
    MASK_FILES = ['*.cpp','*.h','*.txt',]
    DIR_TO = '11'
    DIR_FROM = '.'
    
    def __init__(self, builder):
        self.MASK_FILES=builder.MASK_FILES
        self.DIR_TO=builder.DIR_TO
        self.DIR_FROM=builder.DIR_FROM
        
    def mListFiles(self,):
        l = os.listdir(self.DIR_FROM)
        l2 = [self.DIR_FROM + i for i in l]
        return l2

    def getMask(self,s):
        '''
        s = '*.cpp'
        return: 'cpp'
        '''
        pattern = r'[*].(\w+)$' 
        string = s
        match = re.search(pattern, string)
        s2 = None
        if match:
            if len(match.groups())>0:
                s2 = match.group(1)
        return s2

    def mFindMask(self,s,p):
        '''
        s='file.cpp'
        p ='cpp'
        '''
        pattern = r'.+\.'+ p + '$'
        string = s
        match = re.search(pattern, string)
        s2 = None
        if match:
            s2 = match[0]
        return s2
    
    def mGetList(self):
        '''
        get list of files 
        '''
        files = self.mListFiles()
        files2 = []
        for i in files:
                for j in self.MASK_FILES:
                    m = self.getMask(j)
                    s = self.mFindMask(i,m)
                    if s:
                        files2.append(s)
        return files2
    
    def mFileStrings(self,file):
        l = []
        with open(file,"rb") as fh :
            l = fh.readlines()
        return l
    
    def mQuantityStringsFile(self,file):
        '''
        input: file
        output: quantity of strings in file
        '''
        return len(self.mFileStrings(file))

    def mQuantityStringsFiles(self,files):
        '''
        input: file - list of file
        output: quantity of strings in files(list of file)
        '''
        k=0
        for i in files:
#             print(i)
            k+=self.mQuantityStringsFile(i)
        return k

    def replaceMultiComm(self,l):
        '''
        очистка от комментариев
        input: list
        output: list
        1 - /* ...  */ в одной строке
        2 - /* ...  */ нескольких строках
        3 - //
        '''
    # 1 - /* ...  */ в одной строке    
        i = 0
        for s in l:
            s1 = re.sub(r'\/\*.*\*\/','',s)
            l[i] = s1
            i += 1

    # 2 - /* ...  */ нескольких строках
        flagBeg = False  
        state = 0
        count = 1
        for s in l:
            matchBeg = re.search(r'\/\*.*', s) # /*
            matchEnd = re.search(r'.*\*\/', s) # */
            if state == 1:
                state = 2
            if matchBeg and not flagBeg:
                state = 1
                flagBeg = True
            if matchEnd and flagBeg:
                state = 3
                flagBeg = False
            if state == 1:
                s1 = re.sub(r'\/\*.*','',s)
            elif state == 2:            
                s1 = '\n'
            elif state == 3:            
                s1 = re.sub(r'.*\*\/','',s)
            else:
                s1 = s
            l[count-1] = s1    

            count += 1
            if state == 3:
                state = 0
    # 3 - //    
        i = 0
        for s in l:
            s1 = re.sub(r'\/\/.+','',s)
            l[i] = s1
            i += 1


    def searchRus(self, file):
        '''
        принимает: имя файла
        возвращает: номера строк с русским шрифтом
        1 - открывает файл как список строк
        2 - очищает от комментариев replaceMultiComm(list)
        3 - делает список строк номеров строк с русским шрифтом 
        игнорирует  комментарии
        '''
    #     1 - открывает файл как список строк
        l=[]
        with codecs.open(file,"r",'utf-8') as fh:
            l=fh.readlines()
        flagComm = False    
    #     2 - очищает от комментариев replaceMultiComm(list)
        replaceMultiComm(l)
    #     3 - делает список строк номеров строк с русским шрифтом 
        count = 1
        ln=[]
        for s in l:
            match = re.search(r'[а-яА-Я]', s)
            if match:
                ln.append(count)
    #             print(count)
            count += 1
        return ln
    
    
class Builder:
    MASK_FILES = ['*.cpp','*.h']
    DIR_FROM = ""
    DIR_TO = '..\\tmp1\\'
    
    def setMaskFiles(self, maskFiles):
        self.MASK_FILES = maskFiles
        return self
    
    def setDirTo(self, dirTo):
        self.DIR_TO = dirTo
        return self

    def setDirFrom(self, dirFrom):
        self.DIR_FROM = dirFrom
        return self
    
    def build(self):
         return ClFind(self)

builder = Builder()
builder.setMaskFiles(['*.cpp','*.h','*.ui'])
builder.setMaskFiles(['*.cpp','*.h'])
builder.setDirTo('..\\tmp1\\')
builder.setDirFrom('.\\a01diagcpp\\')

clFind=builder.build()

l=clFind.mGetList()
# print(l)
sz = clFind.mQuantityStringsFiles(l)
count = 0
for i in l:
#     print(i)
    ln = clFind.searchRus(i)
    count += len(ln)
    if len(ln):
        print(i, ln)
print(count)
print(sz)


.\a01diagcpp\diagnostica.cpp [103, 105, 111, 116, 125, 133, 138, 140, 141, 144, 146, 149, 150, 155, 157, 160, 162, 163, 167, 169, 171, 172, 175, 177, 178, 182, 184, 185, 187, 190, 192, 195, 201, 249, 251, 257, 262, 271, 279, 281, 285, 291, 295, 297, 302, 322, 331, 340, 349, 358, 361, 369, 371, 373, 446, 448, 487, 489, 490, 496, 502, 508]
.\a01diagcpp\dialog.cpp [283, 284, 285, 286, 291, 292, 293, 294, 295, 299, 300, 702, 727, 755, 792, 793, 812, 817, 939]
.\a01diagcpp\dialog2.cpp [885, 886, 887, 888, 893, 894, 895, 896, 897, 901, 902, 1235, 1236, 1277, 1278, 1404, 1405, 1455, 1456]
.\a01diagcpp\dlgfault.cpp [10, 12, 13, 14, 21]
.\a01diagcpp\glcu.cpp [180, 181, 210, 211, 222, 223, 228, 229, 245, 251, 257, 263, 269, 296, 298, 304, 309, 318, 326, 328, 332, 338, 342, 344, 349, 369, 378, 387, 396, 405, 408, 416, 418, 420, 493, 495, 534, 536, 537, 543, 549, 555]
.\a01diagcpp\list.cpp [169, 214, 215, 216, 217, 218, 219, 529, 668]
.\a01diagcpp\main.cpp [55]
.\a01diagcpp\tabcust.cpp [15, 16, 17

#### Поиск, замена многострочного комментария

##### удалить многострочный комментарий в файле

In [3]:
import codecs

def delMultComm(file):
    s = ""
    with codecs.open(file,"r","utf-8") as fh: 
        s = fh.read()
    s1 = re.sub(r'\/\*(.|\n|\r)*\*\/','',s)    
    print(s1)

delMultComm("dialog.cpp")    

//dialog.cpp
//пороги
	
	intPressEnter=0;
	widgetRiftZ->setLevelEdgeColor(1,QColor(Qt::red));	//изм.зн.ампл.
	widgetRiftZ->setLevelEdgeColor(2,QColor(Qt::blue));	//уст.пор.зап.
	widgetRiftZ->setLevelEdgeColor(3,QColor(Qt::cyan));	//уст.пор.вид.
QString sss="уст.пор.вид.";

//120	параметры АРП
//		бит 14:	1 — использовать «выбор РЛС»
	fWord(unitReceive->basicSetting[120]);
	if(word.bit14) checkBoxARPUseRLS->setChecked(true);
	else checkBoxARPUseRLS->setChecked(false);
//		бит 15:	склонение (0 — географическое (истинное), 1 — магнитное)
	if(word.bit15)	radioButtonArpGradientMagnetic->setChecked(true);
	else radioButtonArpGradientGeographic->setChecked(true);
//биты 1,0:	номер АРП
	a1 = word.bit1*2+word.bit0;
	comboBoxArpNumber->setCurrentIndex(a1);
//127	порт выхода на осциллограф
//	младший байт: канал 1
//	0: выключен		-	23
//	1: >ВЗ			-	1
//	2: >П			-	2
//	3: >А
//	4: >М
//	5: >С
//	6: З>
//	7: П>
//	8: А>
//	9: М>
//	10: С>
//	11: ПЗК
//	12: Т1
//	13: Т2
//	14: Т3




In [ ]:
        with open(file,"rb") as fh :
            l = fh.readlines()
            
[а-яА-Я]


#### test

In [9]:
import codecs
import re

def searchRus(file):
    '''
    принимает: имя файла
    возвращает: номера строк с русским шрифтом
    игнорирует однострочный комментарий
    '''
    l=[]
    with codecs.open(file,"r",'utf-8') as fh:
        l=fh.readlines()
    flagComm = False    
    count = 1
    ln=[]
    for s in l:
        s2 = re.sub(r'\/\/.+','',s) # //
        s1 = re.sub(r'\/\*.*\*\/','',s2) # /**/
# не считать многострочный комментарий
        match = re.search(r'\/\*.*', s1)
        match = re.search(r'[а-яА-Я]', s1)
#         match = re.search(r'\/\*', s1)
        if match:
            ln.append(count)
#             print(count)
        count += 1
    return ln
ln = searchRus('dialog.cpp')        
print(ln)

[8, 10]


#### searchRus

In [21]:
import codecs
import re

l = ['.\\a01diagcpp\\blocmeas.cpp', '.\\a01diagcpp\\blocmeas.h', '.\\a01diagcpp\\board.cpp', '.\\a01diagcpp\\board.h', '.\\a01diagcpp\\diagnostica.cpp', '.\\a01diagcpp\\diagnostica.h', '.\\a01diagcpp\\diagnosticapsp.cpp', '.\\a01diagcpp\\diagnosticapsp.h', '.\\a01diagcpp\\dialog.cpp', '.\\a01diagcpp\\dialog.h', '.\\a01diagcpp\\dialog.ui', '.\\a01diagcpp\\dialog2.cpp', '.\\a01diagcpp\\dialog3.cpp', '.\\a01diagcpp\\dlgcust.cpp', '.\\a01diagcpp\\dlgcust.h', '.\\a01diagcpp\\dlgcust.ui', '.\\a01diagcpp\\dlgfault.cpp', '.\\a01diagcpp\\dlgfault.h', '.\\a01diagcpp\\dlgfault.ui', '.\\a01diagcpp\\glcu.cpp', '.\\a01diagcpp\\glcu.h', '.\\a01diagcpp\\levels.cpp', '.\\a01diagcpp\\levels.h', '.\\a01diagcpp\\list.cpp', '.\\a01diagcpp\\list.h', '.\\a01diagcpp\\listdlg.cpp', '.\\a01diagcpp\\listdlg.h', '.\\a01diagcpp\\listdlg.ui', '.\\a01diagcpp\\main.cpp', '.\\a01diagcpp\\netexchange.cpp', '.\\a01diagcpp\\netexchange.h', '.\\a01diagcpp\\olineedit.cpp', '.\\a01diagcpp\\olineedit.h', '.\\a01diagcpp\\plineedit.cpp', '.\\a01diagcpp\\plineedit.h', '.\\a01diagcpp\\rastr.cpp', '.\\a01diagcpp\\rastr.h', '.\\a01diagcpp\\tabcust.cpp', '.\\a01diagcpp\\tabcust.h', '.\\a01diagcpp\\tabcust.ui', '.\\a01diagcpp\\tabcust0.cpp', '.\\a01diagcpp\\tabcust0.h', '.\\a01diagcpp\\tabcust0.ui', '.\\a01diagcpp\\tabcust00.cpp', '.\\a01diagcpp\\tabcust2.cpp', '.\\a01diagcpp\\tabcust3.cpp', '.\\a01diagcpp\\tabcustkors.cpp', '.\\a01diagcpp\\tabcustmvrl.cpp', '.\\a01diagcpp\\tabpcm.cpp', '.\\a01diagcpp\\tabpcm.h', '.\\a01diagcpp\\tabpcm.ui', '.\\a01diagcpp\\tabpdo.cpp', '.\\a01diagcpp\\tabpdo.h', '.\\a01diagcpp\\tabpdo.ui', '.\\a01diagcpp\\tabpiu.cpp', '.\\a01diagcpp\\tabpiu.h', '.\\a01diagcpp\\tabpiu.ui', '.\\a01diagcpp\\tabps5.cpp', '.\\a01diagcpp\\tabps5.h', '.\\a01diagcpp\\tabps5.ui', '.\\a01diagcpp\\unit.cpp', '.\\a01diagcpp\\unit.h', '.\\a01diagcpp\\unn.cpp', '.\\a01diagcpp\\unn.h', '.\\a01diagcpp\\widrastr.cpp', '.\\a01diagcpp\\widrastr.h', '.\\a01diagcpp\\widrastr.ui', '.\\a01diagcpp\\widrastrtext.cpp', '.\\a01diagcpp\\widrastrtext.h', '.\\a01diagcpp\\wrd.cpp', '.\\a01diagcpp\\wrd.h']
l = ['.\\a01diagcpp\\blocmeas.cpp', '.\\a01diagcpp\\blocmeas.h', '.\\a01diagcpp\\board.cpp', '.\\a01diagcpp\\board.h', '.\\a01diagcpp\\diagnostica.cpp', '.\\a01diagcpp\\diagnostica.h', '.\\a01diagcpp\\diagnosticapsp.cpp', '.\\a01diagcpp\\diagnosticapsp.h', '.\\a01diagcpp\\dialog.cpp', '.\\a01diagcpp\\dialog.h', '.\\a01diagcpp\\dialog2.cpp', '.\\a01diagcpp\\dialog3.cpp', '.\\a01diagcpp\\dlgcust.cpp', '.\\a01diagcpp\\dlgcust.h', '.\\a01diagcpp\\dlgfault.cpp', '.\\a01diagcpp\\dlgfault.h', '.\\a01diagcpp\\glcu.cpp', '.\\a01diagcpp\\glcu.h', '.\\a01diagcpp\\levels.cpp', '.\\a01diagcpp\\levels.h', '.\\a01diagcpp\\list.cpp', '.\\a01diagcpp\\list.h', '.\\a01diagcpp\\listdlg.cpp', '.\\a01diagcpp\\listdlg.h', '.\\a01diagcpp\\main.cpp', '.\\a01diagcpp\\netexchange.cpp', '.\\a01diagcpp\\netexchange.h', '.\\a01diagcpp\\olineedit.cpp', '.\\a01diagcpp\\olineedit.h', '.\\a01diagcpp\\plineedit.cpp', '.\\a01diagcpp\\plineedit.h', '.\\a01diagcpp\\rastr.cpp', '.\\a01diagcpp\\rastr.h', '.\\a01diagcpp\\tabcust.cpp', '.\\a01diagcpp\\tabcust.h', '.\\a01diagcpp\\tabcust0.cpp', '.\\a01diagcpp\\tabcust0.h', '.\\a01diagcpp\\tabcust00.cpp', '.\\a01diagcpp\\tabcust2.cpp', '.\\a01diagcpp\\tabcust3.cpp', '.\\a01diagcpp\\tabcustkors.cpp', '.\\a01diagcpp\\tabcustmvrl.cpp', '.\\a01diagcpp\\tabpcm.cpp', '.\\a01diagcpp\\tabpcm.h', '.\\a01diagcpp\\tabpdo.cpp', '.\\a01diagcpp\\tabpdo.h', '.\\a01diagcpp\\tabpiu.cpp', '.\\a01diagcpp\\tabpiu.h', '.\\a01diagcpp\\tabps5.cpp', '.\\a01diagcpp\\tabps5.h', '.\\a01diagcpp\\unit.cpp', '.\\a01diagcpp\\unit.h', '.\\a01diagcpp\\unn.cpp', '.\\a01diagcpp\\unn.h', '.\\a01diagcpp\\widrastr.cpp', '.\\a01diagcpp\\widrastr.h', '.\\a01diagcpp\\widrastrtext.cpp', '.\\a01diagcpp\\widrastrtext.h', '.\\a01diagcpp\\wrd.cpp', '.\\a01diagcpp\\wrd.h']

def replaceMultiComm(l):
    '''
    очистка от комментариев
    input: list
    output: list
    1 - /* ...  */ в одной строке
    2 - /* ...  */ нескольких строках
    3 - //
    '''
# 1 - /* ...  */ в одной строке    
    i = 0
    for s in l:
        s1 = re.sub(r'\/\*.*\*\/','',s)
        l[i] = s1
        i += 1
    
# 2 - /* ...  */ нескольких строках
    flagBeg = False  
    state = 0
    count = 1
    for s in l:
        matchBeg = re.search(r'\/\*.*', s) # /*
        matchEnd = re.search(r'.*\*\/', s) # */
        if state == 1:
            state = 2
        if matchBeg and not flagBeg:
            state = 1
            flagBeg = True
        if matchEnd and flagBeg:
            state = 3
            flagBeg = False
        if state == 1:
            s1 = re.sub(r'\/\*.*','',s)
        elif state == 2:            
            s1 = '\n'
        elif state == 3:            
            s1 = re.sub(r'.*\*\/','',s)
        else:
            s1 = s
        l[count-1] = s1    

        count += 1
        if state == 3:
            state = 0
# 3 - //    
    i = 0
    for s in l:
        s1 = re.sub(r'\/\/.+','',s)
        l[i] = s1
        i += 1


def searchRus(file):
    '''
    принимает: имя файла
    возвращает: номера строк с русским шрифтом
    1 - открывает файл как список строк
    2 - очищает от комментариев replaceMultiComm(list)
    3 - делает список строк номеров строк с русским шрифтом 
    игнорирует  комментарии
    '''
#     1 - открывает файл как список строк
    l=[]
    with codecs.open(file,"r",'utf-8') as fh:
        l=fh.readlines()
    flagComm = False    
#     2 - очищает от комментариев replaceMultiComm(list)
    replaceMultiComm(l)
#     3 - делает список строк номеров строк с русским шрифтом 
    count = 1
    ln=[]
    for s in l:
        match = re.search(r'[а-яА-Я]', s)
        if match:
            ln.append(count)
#             print(count)
        count += 1
    return ln

count = 0
for i in l:
#     print(i)
    ln = searchRus(i)
    count+=len(ln)
    if len(ln):
        print(i,ln)
    
print(count)


.\a01diagcpp\diagnostica.cpp [103, 105, 111, 116, 125, 133, 138, 140, 141, 144, 146, 149, 150, 155, 157, 160, 162, 163, 167, 169, 171, 172, 175, 177, 178, 182, 184, 185, 187, 190, 192, 195, 201, 249, 251, 257, 262, 271, 279, 281, 285, 291, 295, 297, 302, 322, 331, 340, 349, 358, 361, 369, 371, 373, 446, 448, 487, 489, 490, 496, 502, 508]
.\a01diagcpp\dialog.cpp [283, 284, 285, 286, 291, 292, 293, 294, 295, 299, 300, 702, 727, 755, 792, 793, 812, 817, 939]
.\a01diagcpp\dialog2.cpp [885, 886, 887, 888, 893, 894, 895, 896, 897, 901, 902, 1235, 1236, 1277, 1278, 1404, 1405, 1455, 1456]
.\a01diagcpp\dlgfault.cpp [10, 12, 13, 14, 21]
.\a01diagcpp\glcu.cpp [180, 181, 210, 211, 222, 223, 228, 229, 245, 251, 257, 263, 269, 296, 298, 304, 309, 318, 326, 328, 332, 338, 342, 344, 349, 369, 378, 387, 396, 405, 408, 416, 418, 420, 493, 495, 534, 536, 537, 543, 549, 555]
.\a01diagcpp\list.cpp [169, 214, 215, 216, 217, 218, 219, 529, 668]
.\a01diagcpp\main.cpp [55]
.\a01diagcpp\tabcust.cpp [15, 16, 17

1849 - c комментариями  
276 - без комментариев  
206 - без многострочного комментария  
206 - внутри класса 

In [11]:
flagBeg = False
for i in range(10):
    if i == 3:
        flag = True
        
    if i == 7:
        flag = False
    print(i,flag)

0 False
1 False
2 False
3 True
4 True
5 True
6 True
7 False
8 False
9 False


In [17]:
import codecs
import re


def searchComm(file):
    '''
    принимает: имя файла
   

    '''
    l=[]
    with codecs.open(file,"r",'utf-8') as fh:
        l=fh.readlines()
    replaceMultiComm(l)    
# end не считать многострочный комментарий
    print(l)

searchComm('dialog.cpp')        


['\n', '\n', '\n', '\n', '\tintPressEnter=0;\n', '\twidgetRiftZ->setLevelEdgeColor(1,QColor(Qt::red));\t\n', '\twidgetRiftZ->setLevelEdgeColor(2,QColor(Qt::blue));\t\n', '\twidgetRiftZ->setLevelEdgeColor(3,QColor(Qt::cyan));\t\n', 'QString sss="уст.пор.вид.";\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\tfWord(unitReceive->basicSetting[120]);\n', '\tif(word.bit14) checkBoxARPUseRLS->setChecked(true);\n', '\telse checkBoxARPUseRLS->setChecked(false);\n', '\n', '\tif(word.bit15)\tradioButtonArpGradientMagnetic->setChecked(true);\n', '\telse radioButtonArpGradientGeographic->setChecked(true);\n', '\n', '\ta1 = word.bit1*2+word.bit0;\n', '\tcomboBoxArpNumber->setCurrentIndex(a1);\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n']


1720 276-11

file:///D:/mailCloud/prjother/083qt/a01diagcpp/  
[regex101 php,python,...](https://regex101.com/)  
  